In [1]:
import pandas as pd
import numpy as np

In [2]:
match_ngo = pd.read_excel("../../orgs/org_list_ngo_check.xlsx",sheet_name="match")
ngo_list = pd.read_excel("../../orgs/org_list_ngo.xlsx")
igo_list = pd.read_excel("../../orgs/org_list_igo.xlsx")
party_list = pd.read_excel("../../orgs/org_list_party.xlsx")
un_list = pd.read_excel("../../orgs/org_list_un.xlsx")
un_list = un_list[un_list["Type"].isna()==False]

In [3]:
energy_speakers = pd.read_excel("manual_match_energy.xlsx")
rename_orgs = pd.read_excel("manual_rename_orgs.xlsx")
video_match = pd.read_csv("cop28_video_match_energy.csv")

In [4]:
for vid in energy_speakers["video_num"].unique():
    this_event = energy_speakers.query("video_num==@vid")
    this_event = this_event[["video_num","organisation"]].drop_duplicates()
    this_event["SE_Code"] = video_match.query("video_num==@vid")["SE_Code"].iloc[0]
    this_event["topics"] = video_match.query("video_num==@vid")["new_topics_ctfidf"].iloc[0]
    
    for i in this_event.index:
        org_original = this_event.loc[i,"organisation"]
        org_clean = org_original
    
        if org_clean in list(party_list["party"]): # ORGANZIED BY A PARTY
            org_type = party_list.query("party==@org_clean")["type"]
            this_event.loc[i,"Org_Type"] = "Party"
            this_event.loc[i,"Country"] = org_clean
        elif org_clean in list(ngo_list["Organization"]): # ORGANIZED BY A NGO
            org_type = ngo_list.query("Organization==@org_clean")["Constituency"].iloc[0]
            country_aff = ngo_list.query("Organization==@org_clean")["Country"].iloc[0]
            this_event.loc[i,"Org_Type"] = org_type
            this_event.loc[i,"Country"] = country_aff
        elif org_clean in list(igo_list["Organization"]): # ORGANIZED BY IGO
            country_aff = igo_list.query("Organization==@org_clean")["Country"].iloc[0]
            this_event.loc[i,"Org_Type"] = "IGO"
            this_event.loc[i,"Country"] = country_aff
        elif org_clean in list(un_list["Organization"]): # ORGANIZED BY THE UN
            this_event.loc[i,"Org_Type"] = "UN"
            this_event.loc[i,"Country"] = "International"
        elif org_clean in list(rename_orgs["Name"]):
            this_event.loc[i,"Org_Type"] = rename_orgs.query("Name==@org_clean")["Org_Type"].iloc[0]
    
    for j in np.arange(len(this_event))[:-1]:
        this_event_select = this_event.iloc[j+1:]
        this_event_select = this_event_select.rename(columns={"organisation":"target_org"})
        this_event_select["source_org"] =  this_event.iloc[j]["organisation"]
        if j == 0:
            this_event_edges = this_event_select
        else:
            this_event_edges = pd.concat([this_event_edges,this_event_select])
    this_event_edges["Weight"] = 1/(np.arange(len(this_event)).sum())
    
    if vid == energy_speakers["video_num"].unique()[0]:
        energy_speakers_clean = this_event
        energy_speakers_edges = this_event_edges
    else:
        energy_speakers_clean = pd.concat([energy_speakers_clean,this_event])
        energy_speakers_edges = pd.concat([energy_speakers_edges,this_event_edges])

energy_speakers_edges = energy_speakers_edges.drop(columns=["Org_Type","Country"])
HRW_index = energy_speakers_clean.query("organisation=='Human Rights Watch, Inc. (HRW)'").index
energy_speakers_clean.loc[HRW_index,"Org_Type"] = "NGO"
energy_speakers_clean

,video_num,organisation,SE_Code,topics,Org_Type,Country
0,337,Oil Change International (OCI),COP28-019,19,Constituency - Environmental CAN,United States of America
1,337,Climate Action Network International (CAN Inte...,COP28-019,19,Constituency - Environmental CAN,Germany
2,337,Climate Analytics GmbH,COP28-019,19,Constituency - Research and independent,Germany
3,337,Indigenous Environmental Network,COP28-019,19,Civil Society Campaign,NaN
4,337,Fossil Fuel Non-Proliferation Treaty Initiative,COP28-019,19,Civil Society Campaign,NaN
...,...,...,...,...,...,...
156,25,Fédération des travailleurs et des travailleus...,COP28-309,19,Constituency - Trade Unions,Canada
157,58,Climate Action Network International (CAN Inte...,COP28-339,19,Constituency - Environmental CAN,Germany
158,58,International Institute for Sustainable Develo...,COP28-339,19,Constituency - Research and independent,Canada
159,58,International Trade Union Confederation (ITUC),COP28-339,19,Constituency - Trade Unions,Belgium


In [5]:
energy_speakers_edges

,video_num,target_org,SE_Code,topics,source_org,Weight
1,337,Climate Action Network International (CAN Inte...,COP28-019,19,Oil Change International (OCI),0.035714
2,337,Climate Analytics GmbH,COP28-019,19,Oil Change International (OCI),0.035714
3,337,Indigenous Environmental Network,COP28-019,19,Oil Change International (OCI),0.035714
4,337,Fossil Fuel Non-Proliferation Treaty Initiative,COP28-019,19,Oil Change International (OCI),0.035714
5,337,International Indigenous Forum of Climate Change,COP28-019,19,Oil Change International (OCI),0.035714
...,...,...,...,...,...,...
159,58,International Trade Union Confederation (ITUC),COP28-339,19,Climate Action Network International (CAN Inte...,0.166667
160,58,Fossil Fuel Non-Proliferation Treaty Initiative,COP28-339,19,Climate Action Network International (CAN Inte...,0.166667
159,58,International Trade Union Confederation (ITUC),COP28-339,19,International Institute for Sustainable Develo...,0.166667
160,58,Fossil Fuel Non-Proliferation Treaty Initiative,COP28-339,19,International Institute for Sustainable Develo...,0.166667


In [6]:
energy_speaker_nodes = energy_speakers_clean[["organisation","Org_Type","Org_Type","Country"]].drop_duplicates()
energy_speaker_nodes.index.names = ['ID']
energy_speaker_nodes = energy_speaker_nodes.reset_index()
energy_speaker_nodes.to_excel("speaker_nodes_energy.xlsx",index=False)

In [7]:
energy_speakers_edges_labeled = pd.merge(energy_speakers_edges, energy_speaker_nodes[["organisation","ID"]], 
                                         left_on = "target_org",right_on="organisation", how="left")
energy_speakers_edges_labeled = energy_speakers_edges_labeled.rename(columns={"ID":"Target"}).drop(columns="organisation")
energy_speakers_edges_labeled = pd.merge(energy_speakers_edges_labeled, energy_speaker_nodes[["organisation","ID"]], 
                                         left_on = "source_org",right_on="organisation", how="left")
energy_speakers_edges_labeled = energy_speakers_edges_labeled.rename(columns={"ID":"Source"}).drop(columns="organisation")
energy_speakers_edges_labeled.to_excel("speaker_edges_energy.xlsx",index=False)